In [1]:
import time
import re
from selenium import webdriver
import csv
from bs4 import BeautifulSoup as bs

browser = webdriver.Chrome()

# Tell Selenium to get the URL you're interested in.
browser.get("https://statestreet.wd1.myworkdayjobs.com/Global")
# browser.implicitly_wait(100)

# Selenium script to scroll to the bottom, wait 3 seconds for the next batch of data to load, then continue scrolling.  It will continue to do this until the page stops loading new data.
lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
    lastCount = lenOfPage
    time.sleep(3)
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount==lenOfPage:
#        print('lastCount and lenOfPage matched: ',lenOfPage)
        match=True

# Now that the page is fully scrolled, grab the source code.
source_data = browser.page_source

# Throw your source into BeautifulSoup and start parsing!
bs_data = bs(source_data, "lxml")
#print(bs_data)

base_url = 'https://statestreet.wd1.myworkdayjobs.com/en-US/Global/job/'
test_links = bs_data.find_all('li',attrs={'class':'WIYF WK3N WE5 WP-F'})
# my_str = "hey th~!ere"
# my_new_string = re.sub('[^a-zA-Z0-9 \n\.]', '', my_str)
# print my_new_string
link_url =[]

for link in test_links:
    designation = link.find('div',attrs={'class':'gwt-Label WOTO WISO'})
    address = link.find('span', attrs={'class':'gwt-InlineLabel WM-F WLYF'})
#    print('designation is: ', designation.get_text())
#    print('address is: ', address.get_text())
    address_label = address.get_text()
    title = designation.get_text()

    address_label_contents = address_label.split('|')
#    print(address_label_contents)
#    print(type(address_label_contents))
    a = len(address_label_contents)
#    print(a)
    if a ==3:
        location_more = address_label_contents[0].strip()
        #        print(location_more)
        if location_more.count(',')>1:
            location_trim = location_more.rsplit(',',1)
            location = location_trim[0]
        else:
            location = location_more
        location = re.sub('[^a-zA-Z0-9-]+','-',location)
#        print(location)
        title = re.sub('[^a-zA-Z0-9-]','-',title)
        job_id = address_label_contents[1].strip()
#     location.replace(',','-')
#     print(location, job_id)
        if len(address_label_contents)==2:
            url = base_url+title+'_'+address_label_contents[0].strip()
        else:
            url = base_url+location+'/'+title+'_'+job_id
            link_url.append(url)
#        print(url)
browser.close()

In [2]:
import collections
def get_count(word_count_tuple):
    return word_count_tuple[1]

word_counts = collections.Counter(link_url)

# This function sorts the words based on the number of times repeated and saves them in a csv file
def print_top():
    word_count = word_counts
    items = sorted(word_count.items(),key=get_count,reverse=True)
    with open('/Users/rajsharavan/Desktop/Python/url_state_street.csv','w') as out:
        csv_out=csv.writer(out)
        #csv_out.writerow('Name')
        for row in items:
            csv_out.writerow(row)
print_top()

In [ ]:
des = 'Account Manager Assistant, Commercial Lines'
des = re.sub('[^a-zA-Z0-9-]','-',des)
#print(des)

In [ ]:
#This checks whether the URL is working fine or not
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import re
from selenium import webdriver

browser = webdriver.Chrome()
#browser.get("https://bbt.wd1.myworkdayjobs.com/en-US/Careers/job/Frederick-MD/Insurance-Commercial-Account-Manager-I_R0013213")
#browser.title

contents = []
contents1 = []
with open('/Users/rajsharavan/Desktop/Python/url_state_street.csv','r') as csvf: # Open file in read mode
    urls = csv.reader(csvf)
    for url in urls:
        contents.append(url[0]) # Add each url to list contents
#print(contents[:10])

for url in contents:
    browser.get(url)
    time.sleep(2)
   # print(browser.title)

browser.close()